In [2]:
import pandas as pd

In [3]:
fs_df = pd.read_csv('../data/fullsample.csv')
fs_df.shape

(7395885, 12)

In [27]:
fs_df.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


In [16]:
row = df.iloc(0)[0:1]

In [20]:
pd.to_datetime(row['BEGIN'], format='%Y-%m-%dT%H:%M:%S')

0   2021-07-31 22:15:00
Name: BEGIN, dtype: datetime64[ns]

In [ ]:
# not sure how long this takes, didn't wait
# df['BEGIN'] = df['BEGIN'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S'))

In [22]:
ce5_df = pd.read_csv('../data/cs5_9024_sbatch.csv')

In [25]:
ce5_df.head(-5)

,0,1,2,3,4,5,6
0,2020-10-16 08:34:42.779719,user 9204,retry 0,time 5.240251064300537,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5590dbe24f10.2098696...
1,2020-10-16 08:37:40.922337,user 9201,retry 0,time 0.2213442325592041,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5590daaa31b0.2098696...
2,2020-10-16 08:53:15.711346,user 9204,retry 0,time 0.14244365692138672,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5590d7f1d1d0.2098696...
3,2020-10-16 09:03:14.279817,user 9203,retry 0,time 0.9998009204864502,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5590daaad040.2098696...
4,2020-10-16 09:04:56.472464,user 9204,retry 0,time 0.0663447380065918,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5590d8ce0c20.2098696...
...,...,...,...,...,...,...,...
32178,2021-10-07 19:45:07.554477,user 9204,retry 0,time 0.030558109283447266,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x555675035ec0.1189023...
32179,2021-10-07 19:53:54.234049,user 9204,retry 0,time 0.03154706954956055,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x55567364a4d0.1189023...
32180,2021-10-07 20:14:58.603967,user 9204,retry 0,time 0.03008270263671875,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x555675035ec0.1189023...
32181,2021-10-07 20:23:46.239798,user 9204,retry 0,time 0.028536319732666016,returncode 0,command ['/usr/bin/sbatch','/tmp/condor_g_scratch.0x5556742688d0.1189023...
